In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2
import torchvision.utils as utils
import copy

import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import ml_library as ML

In [ ]:
history = {
    'model': str,
    'batch_size': int,
    'num_workers': int,
    'learning_rate': float,
    'momentum': None,
    'optimizer': str,
    'training_loss': [],
    'validation_loss': [],
    'training_accuracy': [],
    'validation_accuracy': []
}

# **Lab B3:** Medical Imaging Analysis

In this lab you will set up, and train a model that seeks to perform classification of medical OCT scans. In the data there are four classes of scans: Healthy, CNV, DME and DRUSEN. The last three are eye diseases that cause visible damage to the retina and can be spotted through the OCT scans.

Your job is to set up the tranforms, the model, fit function, test function and the parameters. You must also tune the parameters and add more transforms to reach best performance possible.

IMPORTANT: The data is divided in test, validation and train sets. Make sure to use all three sets properly.

In [ ]:
classes = [0, 1, 2, 3]
class_labels = ['CNV', 'DME', 'DRUSEN', 'NORMAL']

In [ ]:
path_to_data = 'data/'
path_train = path_to_data + 'train'
path_test = path_to_data + 'test'
path_val = path_to_data + 'val'

Source:

- https://debuggercafe.com/pytorch-imagefolder-for-training-cnn-models/

**First Step:** Load the Data, setup Transforms, and Initialize the Dataloader


[Here](https://pytorch.org/vision/stable/transforms.html) is a great resource to learn more about the different transforms that can be added. The goal of the transform is to properly prepare the data to be sent to the model and to add data augmentation. You may have pictures of different resolution sizes, so here is a good time to set a transform to make the sizes of images uniform.

In [ ]:
size = (224, 224)

# Transforms for train set
train_transform = v2.Compose([
    v2.RandomResizedCrop(size=size, antialias=True),
    # v2.Resize(size=size),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.5),
    v2.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    # v2.ElasticTransform(),
    # v2.RandomRotation(degrees=(30, 70)),
    v2.ToTensor(),
    # v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Transforms for test/val set
test_val_transform = v2.Compose([
    v2.Resize(size=size),
    v2.ToTensor(),
    # v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                 ])

In [ ]:
dataset_train = ImageFolder(root=path_train, transform=train_transform)
dataset_test = ImageFolder(root=path_test, transform=test_val_transform)
dataset_val = ImageFolder(root=path_val, transform=test_val_transform)

In [ ]:
# Batch Size
batch_size = 32

# Number of workers
num_workers = 4

In [ ]:
#Initialize dataloader for train set
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size = batch_size, shuffle=True, 
                                             pin_memory = True,
                                             num_workers=num_workers,
                                             persistent_workers=True)

#Initialize dataloader for test set
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size = batch_size, shuffle=False)

#Initialize dataloader for val set
val_loader = torch.utils.data.DataLoader(dataset_val, batch_size = batch_size, shuffle=True)

kfold_dataset = torch.utils.data.ConcatDataset([dataset_train, dataset_val])

In [ ]:
# Helper function to show a batch
def show_batch(sample_batched):
    images_batch, labels_batch = \
            sample_batched[0], sample_batched[1]

    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    print(' '.join('%d' % labels_batch[j] for j in range(batch_size)))

for i_batch, sample_batched in enumerate(train_loader):
    print(i_batch, sample_batched[0].size(),
          sample_batched[1].size())

    # observe 4th batch and stop.
    if i_batch == 3:
        plt.figure()
        show_batch(sample_batched)
        plt.axis('off')
        plt.ioff()
        plt.show()

        
        break

In [ ]:
images_batch, labels_batch = \
            sample_batched[0], sample_batched[1]

**Second Step:** Design Model's Architecture and code it here in with PyTorch.



In [ ]:
torch.manual_seed(0)

# Instantiate Pretrained ResNet
# model = models.resnet50()
model = models.resnet18()

# Modify final layer
# model.fc = nn.Linear(in_features=2048, out_features=(len(classes)), bias=True)
model.fc = nn.Linear(in_features=512, out_features=(len(classes)), bias=True)

# Load onto GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

# View Model to validate
print(model)

**Third Step:** Code Fit and Test functions. This is similar to Lab 3, but this time make sure to use the validation set as well.

Define in `ml_library.py`.

**Fourth Step:** Set Parameters and run model.


In [ ]:
# Define Parameters
num_epochs = 20
lr = 0.0001
momentum = 0.9
hist = copy.deepcopy(history)

In [ ]:
# Define Loss Fuction
loss_function = nn.CrossEntropyLoss()

# Define Optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# hist['momentum'] = momentum
hist['optimizer'] = 'Adam'

hist['model'] = 'ResNet18'
hist['batch_size'] = batch_size
hist['num_workers'] = num_workers
hist['learning_rate'] = lr

In [ ]:
# hist = ML.train(model=model, loss_fn=loss_function, optimizer=optimizer, 
#                 train_loader=train_loader, test_loader=val_loader, num_epochs=num_epochs,
#                 device = device, history=hist)

In [ ]:
# Train model with K-Fold Cross Validation
k_folds = 5

hist = ML.kFoldCrossValTrain(model=model, loss_fn=loss_function, optimizer=optimizer, dataset=kfold_dataset, num_epochs=num_epochs, k_folds=k_folds, device=device, history=hist)

In [ ]:
# Plot Learning Curves
ML.plot_learning_curve(hist)

In [ ]:
print('last validation data accuracy', hist['validation_accuracy'][-1])

In [ ]:
print('highest validation accuracy achieved: ' , max(hist['validation_accuracy']))

In [ ]:
hist['validation_accuracy'].index(max(hist['validation_accuracy']))

In [ ]:
predictions, test_labels = ML.predict(model=model, test_loader=test_loader, device=device)

In [ ]:
cm = confusion_matrix(test_labels, predictions, normalize='all')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)

fig, ax = plt.subplots(figsize=(5, 5))
disp.plot(include_values=True, xticks_rotation='vertical', ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
print('test data accuracy: ', ML.test_accuracy(model=model, data_loader=test_loader, device=device))

In [ ]:
##### LAST STEP, SAVE MODEL
path_model_save = #choose path to save model
torch.save(net.state_dict(), path_model_save)

**Fifth Step:** Model interpretability.

For this assignment, you will interpret the model's results through the use of saliency mapping. You will use the following package: [GitHub](https://github.com/jacobgil/pytorch-grad-cam).

You are expected to install the package on your environment and go through the GitHub to learn its application. Below is an example code to help you get started:

In [ ]:
### Sample Use with ResNet50:

from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50

model = resnet50(pretrained=True)
target_layers = [model.layer4[-1]]


input_tensor = # Your input data


# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=False)

targets = ### your label

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)